In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Data Viz & Regular Expression Libraries :

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# Scikit-Learn ML Libraries :

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report

# Boosting Algorithm Libraries :

# from xgboost                          import XGBClassifier
# from catboost                         import CatBoostClassifier
# from lightgbm                         import LGBMClassifier
from sklearn.ensemble                 import RandomForestClassifier, VotingClassifier
from sklearn.metrics                  import accuracy_score
from sklearn.model_selection          import StratifiedKFold,KFold, train_test_split
from sklearn.linear_model import LogisticRegression
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/processed/train.csv')


# Reduce memory 

In [3]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [4]:

df = reduce_memory_usage(df)

Mem. usage decreased to 781.94 Mb (53.6% reduction)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4606311 entries, 0 to 4606310
Data columns (total 48 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   date                                  object 
 1   customer_id                           int32  
 2   employee_index                        object 
 3   country_of_residence                  object 
 4   gender                                object 
 5   age                                   object 
 6   registration_date                     object 
 7   new_customer                          float16
 8   seniority                             object 
 9   primary_customer                      float16
 10  last_primary_date                     object 
 11  customer_type                         object 
 12  relation_type                         object 
 13  residence_index                       object 
 14  foreigner_index                       object 
 15  spouse_index   

# Clean dataset

In [6]:
def clean_dataset(df):
    """
    Clean and preprocess the banking dataset
    
    Parameters:
    df: pandas DataFrame - Raw dataset
    
    Returns:
    df: pandas DataFrame - Cleaned dataset
    """
    
    print("🧹 CLEANING DATASET...")
    print("="*40)
    
    # =============================
    # 1. HANDLE MISSING VALUES
    # =============================
    print("1️⃣ Handling missing values...")
    
    # Fill missing values for payroll indicators
    # These columns indicate if customer has payroll/pension products
    missing_before = df[['payroll_final_label', 'pensions_2_final_label']].isnull().sum()
    df.fillna(value={
        'payroll_final_label': 0,
        'pensions_2_final_label': 0
    }, inplace=True)
    
    print(f"   ✅ Filled payroll_final_label: {missing_before['payroll_final_label']} missing → 0")
    print(f"   ✅ Filled pensions_2_final_label: {missing_before['pensions_2_final_label']} missing → 0")
    
    # =============================
    # 2. CREATE CUSTOMER TENURE FEATURE
    # =============================
    print("\n2️⃣ Creating customer tenure feature...")
    
    # Convert date columns to datetime
    df['date'] = pd.to_datetime(df['date'])
    df['registration_date'] = pd.to_datetime(df['registration_date'])
    
    # Calculate days since registration (customer tenure)
    days_column = (df['date'] - df['registration_date']).dt.days
    
    # Insert the new column at position 6
    df.insert(loc=6, column='customer_tenure_days', value=days_column)
    
    print(f"   ✅ Created 'customer_tenure_days' feature")
    print(f"   📊 Range: {days_column.min()} to {days_column.max()} days")
    
    # Drop the original registration_date column to save memory
    df.drop(columns=['registration_date'], inplace=True)
    print(f"   🗑️ Dropped 'registration_date' column")
    
    # =============================
    # 3. CONVERT LAST_PRIMARY_DATE TO BINARY INDICATOR
    # =============================
    print("\n3️⃣ Converting last_primary_date to binary indicator...")
    
    # Convert last_primary_date to binary: 1 if date exists, 0 if null
    # This indicates if customer was ever a primary customer
    original_nulls = df['last_primary_date'].isnull().sum()
    df['was_primary_customer'] = df['last_primary_date'].apply(
        lambda x: 1 if pd.notnull(x) else 0
    )
    
    print(f"   ✅ Created 'was_primary_customer' binary feature")
    print(f"   📊 {original_nulls:,} nulls → 0, {len(df) - original_nulls:,} dates → 1")
    
    # Drop the original last_primary_date column
    df.drop(columns=['last_primary_date'], inplace=True)
    print(f"   🗑️ Dropped 'last_primary_date' column")
    
    # =============================
    # 4. REMOVE CONSTANT/DUPLICATE COLUMNS
    # =============================
    print("\n4️⃣ Removing constant and duplicate columns...")
    
    # Remove address_type if it has the same value for all customers
    if 'address_type' in df.columns:
        unique_values = df['address_type'].nunique()
        if unique_values <= 1:
            df.drop(columns=['address_type'], inplace=True)
            print(f"   🗑️ Dropped 'address_type' (constant value)")
        else:
            print(f"   ✅ Kept 'address_type' ({unique_values} unique values)")
    
    # Remove province_code as it's duplicate of province_name
    if 'province_code' in df.columns and 'province_name' in df.columns:
        df.drop(columns=['province_code'], inplace=True)
        print(f"   🗑️ Dropped 'province_code' (duplicate of province_name)")
    
    # =============================
    # 5. CLEAN NUMERIC COLUMNS
    # =============================
    print("\n5️⃣ Cleaning numeric columns...")
    
    # Convert age column - handle 'NA' strings
    if 'age' in df.columns:
        age_before = df['age'].dtype
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
        age_nulls = df['age'].isnull().sum()
        print(f"   ✅ Cleaned 'age': {age_before} → numeric ({age_nulls:,} nulls)")
    
    # Convert seniority column - handle 'NA' strings and negative values
    if 'seniority' in df.columns:
        seniority_before = df['seniority'].dtype
        df['seniority'] = pd.to_numeric(df['seniority'], errors='coerce')
        
        # Handle special negative values (often -999999 means missing)
        negative_values = (df['seniority'] < 0).sum()
        if negative_values > 0:
            df['seniority'] = df['seniority'].where(df['seniority'] >= 0, np.nan)
            print(f"   ⚠️ Converted {negative_values:,} negative seniority values to NaN")
        
        seniority_nulls = df['seniority'].isnull().sum()
        print(f"   ✅ Cleaned 'seniority': {seniority_before} → numeric ({seniority_nulls:,} nulls)")
    
    # =============================
    # 6. SUMMARY STATISTICS
    # =============================
    print("\n📊 CLEANUP SUMMARY:")
    print("-" * 40)
    print(f"   Final shape: {df.shape}")
    print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    print(f"   Null values: {df.isnull().sum().sum():,}")
    
    # Show data types
    print(f"\n📋 DATA TYPES AFTER CLEANUP:")
    for dtype in df.dtypes.value_counts().index:
        count = df.dtypes.value_counts()[dtype]
        print(f"   {dtype}: {count} columns")
    
    return df


In [7]:
df = clean_dataset(df)

🧹 CLEANING DATASET...
1️⃣ Handling missing values...
   ✅ Filled payroll_final_label: 0 missing → 0
   ✅ Filled pensions_2_final_label: 0 missing → 0

2️⃣ Creating customer tenure feature...
   ✅ Created 'customer_tenure_days' feature
   📊 Range: -3.0 to 7498.0 days
   🗑️ Dropped 'registration_date' column

3️⃣ Converting last_primary_date to binary indicator...
   ✅ Created 'was_primary_customer' binary feature
   📊 4,600,165 nulls → 0, 6,146 dates → 1
   🗑️ Dropped 'last_primary_date' column

4️⃣ Removing constant and duplicate columns...
   🗑️ Dropped 'address_type' (constant value)
   🗑️ Dropped 'province_code' (duplicate of province_name)

5️⃣ Cleaning numeric columns...
   ✅ Cleaned 'age': object → numeric (27,734 nulls)
   ⚠️ Converted 14 negative seniority values to NaN
   ✅ Cleaned 'seniority': object → numeric (27,748 nulls)

📊 CLEANUP SUMMARY:
----------------------------------------
   Final shape: (4606311, 46)
   Memory usage: 2935.7 MB
   Null values: 5,979,487

📋 DATA T

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4606311 entries, 0 to 4606310
Data columns (total 46 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   date                                  datetime64[ns]
 1   customer_id                           int32         
 2   employee_index                        object        
 3   country_of_residence                  object        
 4   gender                                object        
 5   age                                   float64       
 6   customer_tenure_days                  float64       
 7   new_customer                          float16       
 8   seniority                             float64       
 9   primary_customer                      float16       
 10  customer_type                         object        
 11  relation_type                         object        
 12  residence_index                       object        
 13  foreigner_in

# Filter data for TypeI 

In [9]:
payment_account_labels = [
    'current_accounts_final_label',
    'payroll_accounts_final_label',
    'junior_accounts_final_label',
    'more_particular_accounts_final_label',
    'particular_accounts_final_label',
    'particular_plus_accounts_final_label',
    'home_account_final_label',
    'payroll_final_label',
    'e_account_final_label'
]

customer_features = [
    'date', 'customer_id', 'employee_index', 'country_of_residence', 'gender',
    'age', 'customer_tenure_days', 'seniority', 'residence_index',  # Added customer_tenure_days
    'foreigner_index', 'spouse_index', 'channel', 'deceased_index', 
    'province_name', 'segment', 'was_primary_customer'  # Added was_primary_customer
]


In [10]:
def filter_data(df) :
    # Lấy khách hàng không có tài khoản thanh toán nào (không có giá trị -1)
    mask = ~(df[payment_account_labels] == -1).any(axis=1)
    
    # Các cột cần giữ lại
    columns_to_keep = customer_features + payment_account_labels
    
    # Lọc dữ liệu
    df = df.loc[mask, columns_to_keep]
    
    print(f"Tổng số khách hàng: {len(df):,}")
    print(f"Khách hàng KHÔNG sở hữu tài khoản thanh toán nào: {len(df):,}")
    
    # Kiểm tra distribution
    print("\n📊 Distribution check:")
    for col in payment_account_labels:
        unique_vals = df[col].unique()
        print(f"  {col}: {unique_vals}")
    
    return df

In [11]:
df = filter_data(df)

Tổng số khách hàng: 442,736
Khách hàng KHÔNG sở hữu tài khoản thanh toán nào: 442,736

📊 Distribution check:
  current_accounts_final_label: [0 1]
  payroll_accounts_final_label: [0 1]
  junior_accounts_final_label: [0 1]
  more_particular_accounts_final_label: [0 1]
  particular_accounts_final_label: [0 1]
  particular_plus_accounts_final_label: [0 1]
  home_account_final_label: [0]
  payroll_final_label: [0 1]
  e_account_final_label: [0 1]


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 442736 entries, 3 to 4606305
Data columns (total 25 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   date                                  442736 non-null  datetime64[ns]
 1   customer_id                           442736 non-null  int32         
 2   employee_index                        415002 non-null  object        
 3   country_of_residence                  415002 non-null  object        
 4   gender                                414987 non-null  object        
 5   age                                   415002 non-null  float64       
 6   customer_tenure_days                  415002 non-null  float64       
 7   seniority                             414995 non-null  float64       
 8   residence_index                       415002 non-null  object        
 9   foreigner_index                       415002 non-null  object  

# Clean memory 

In [13]:

import gc

def cleanup_memory():
    """
    Clean up unused memory and optimize DataFrame
    """
    print("🧹 MEMORY CLEANUP...")
    print("="*40)
    
    # Force garbage collection
    collected = gc.collect()
    print(f"   🗑️ Garbage collected: {collected} objects")
    
    # Get memory info
    import psutil
    import os
    
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    memory_mb = memory_info.rss / 1024 / 1024
    
    print(f"   💾 Current memory usage: {memory_mb:.1f} MB")
    
    return memory_mb

memory_before = cleanup_memory()

🧹 MEMORY CLEANUP...
   🗑️ Garbage collected: 0 objects
   💾 Current memory usage: 417.2 MB


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 442736 entries, 3 to 4606305
Data columns (total 25 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   date                                  442736 non-null  datetime64[ns]
 1   customer_id                           442736 non-null  int32         
 2   employee_index                        415002 non-null  object        
 3   country_of_residence                  415002 non-null  object        
 4   gender                                414987 non-null  object        
 5   age                                   415002 non-null  float64       
 6   customer_tenure_days                  415002 non-null  float64       
 7   seniority                             414995 non-null  float64       
 8   residence_index                       415002 non-null  object        
 9   foreigner_index                       415002 non-null  object  

In [15]:
# Cột target (label) - tài khoản thanh toán
tar_cols = [
    'current_accounts_final_label',
    'payroll_accounts_final_label',
    'junior_accounts_final_label',
    'more_particular_accounts_final_label',
    'particular_accounts_final_label',
    'particular_plus_accounts_final_label',
    'home_account_final_label',
    'payroll_final_label',
    'e_account_final_label'
]

# Cột số (numeric)
num_cols = [
    'age',
    'seniority',
    'new_customer',
    'address_type',
    'province_code',
    'activity_index',
    'income'
]

# Cột phân loại (categorical / object / category)
cat_cols = [
    'employee_index',
    'country_of_residence',
    'gender',
    'primary_customer',
    'customer_type',
    'relation_type',
    'residence_index',
    'foreigner_index',
    'spouse_index',
    'channel',
    'deceased_index',
    'province_name',
    'segment'
]


# Feature Engineering

In [16]:
def create_enhanced_features(df):
    df_enhanced = df.copy()
    
    print("🔧 CREATING ENHANCED FEATURES...")
    print("="*50)
    
    # Check available columns first
    available_cols = df_enhanced.columns.tolist()
    print(f"📋 Available columns: {len(available_cols)}")
    
    # =============================
    # 1. TIME-BASED FEATURES
    # =============================
    print("📅 Creating time-based features...")
    
    # Extract từ date column
    if 'date' in df_enhanced.columns:
        df_enhanced['year'] = df_enhanced['date'].dt.year
        df_enhanced['month'] = df_enhanced['date'].dt.month
        df_enhanced['quarter'] = df_enhanced['date'].dt.quarter
        df_enhanced['day_of_week'] = df_enhanced['date'].dt.dayofweek
        df_enhanced['is_weekend'] = (df_enhanced['day_of_week'] >= 5).astype(int)
        df_enhanced['is_month_end'] = (df_enhanced['date'].dt.day >= 25).astype(int)
        df_enhanced['is_quarter_end'] = df_enhanced['date'].dt.month.isin([3, 6, 9, 12]).astype(int)
    
    # Customer tenure features (using existing customer_tenure_days)
    if 'customer_tenure_days' in df_enhanced.columns:
        df_enhanced['years_since_registration'] = df_enhanced['customer_tenure_days'] / 365.25
        
        # Customer tenure categories
        df_enhanced['tenure_category'] = pd.cut(
            df_enhanced['customer_tenure_days'],
            bins=[-1, 90, 365, 1095, 2190, np.inf],
            labels=['Very_New', 'New', 'Medium', 'Long', 'Very_Long']
        )
    
    # =============================
    # 2. DEMOGRAPHIC FEATURES
    # =============================
    print("👥 Creating demographic features...")
    
    # Age-based features
    if 'age' in df_enhanced.columns:
        df_enhanced['age_group'] = pd.cut(
            df_enhanced['age'],
            bins=[0, 25, 35, 45, 55, 65, 100],
            labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
        )
        
        age_young = (df_enhanced['age'] >= 18) & (df_enhanced['age'] <= 30)
        age_middle = (df_enhanced['age'] >= 31) & (df_enhanced['age'] <= 50)
        age_senior = (df_enhanced['age'] >= 60)
        
        df_enhanced['is_young_adult'] = age_young.fillna(False).astype(int)
        df_enhanced['is_middle_aged'] = age_middle.fillna(False).astype(int)
        df_enhanced['is_senior'] = age_senior.fillna(False).astype(int)
        df_enhanced['age_squared'] = df_enhanced['age'] ** 2
    
    # =============================
    # 3. BANKING RELATIONSHIP FEATURES  
    # =============================
    print("🏦 Creating banking relationship features...")
    
    # Seniority-based features
    if 'seniority' in df_enhanced.columns:
        df_enhanced['seniority_years'] = df_enhanced['seniority'] / 12
        df_enhanced['seniority_category'] = pd.cut(
            df_enhanced['seniority'],
            bins=[-1, 0, 6, 12, 24, 60, np.inf],
            labels=['New', 'Very_Short', 'Short', 'Medium', 'Long', 'Very_Long']
        )
        
        seniority_new = df_enhanced['seniority'] <= 6
        seniority_established = df_enhanced['seniority'] >= 24
        
        df_enhanced['is_new_relationship'] = seniority_new.fillna(False).astype(int)
        df_enhanced['is_established_relationship'] = seniority_established.fillna(False).astype(int)
    
    # Customer status features (check if columns exist)
    df_enhanced['is_employee'] = 0
    if 'employee_index' in df_enhanced.columns:
        df_enhanced['is_employee'] = (df_enhanced['employee_index'] == 1).astype(int)
    
    df_enhanced['is_primary_customer_flag'] = 0  
    if 'was_primary_customer' in df_enhanced.columns:
        df_enhanced['is_primary_customer_flag'] = df_enhanced['was_primary_customer']
    
    # Geographic features
    if 'country_of_residence' in df_enhanced.columns:
        domestic = df_enhanced['country_of_residence'] == 'ES'
        df_enhanced['is_domestic'] = domestic.fillna(False).astype(int)
    
    if 'foreigner_index' in df_enhanced.columns:
        foreigner = df_enhanced['foreigner_index'] == 1
        df_enhanced['is_foreigner'] = foreigner.fillna(False).astype(int)
    

    # =============================
    # 5. INTERACTION FEATURES
    # =============================
    print("🔗 Creating interaction features...")
    
    # Age-Seniority interactions
    if 'age' in df_enhanced.columns and 'seniority' in df_enhanced.columns:
        age_filled = df_enhanced['age'].fillna(0)
        seniority_filled = df_enhanced['seniority'].fillna(0)
        
        df_enhanced['age_seniority_interaction'] = (age_filled * seniority_filled) / 100
        df_enhanced['seniority_per_age'] = seniority_filled / (age_filled + 1)
    
    # Age-Tenure interactions
    if 'age' in df_enhanced.columns and 'customer_tenure_days' in df_enhanced.columns:
        age_filled = df_enhanced['age'].fillna(0)
        tenure_filled = df_enhanced['customer_tenure_days'].fillna(0)
        
        df_enhanced['age_tenure_ratio'] = age_filled / (tenure_filled/365 + 1)
    
    # =============================
    # 6. BEHAVIORAL FEATURES
    # =============================
    print("🎯 Creating behavioral features...")
    
    # Channel preference
    if 'channel' in df_enhanced.columns:
        channel_mapping = {
            'KAT': 'Traditional',
            'KFC': 'Phone', 
            'KHE': 'Digital',
            'KHM': 'Mobile',
            'KHN': 'Online'
        }
        df_enhanced['channel_type'] = df_enhanced['channel'].map(channel_mapping).fillna('Other')
        df_enhanced['is_digital_channel'] = df_enhanced['channel_type'].isin(['Digital', 'Mobile', 'Online']).astype(int)
    
    # Customer segment enhancement
    if 'segment' in df_enhanced.columns:
        df_enhanced['is_vip_segment'] = df_enhanced['segment'].str.contains('VIP', na=False).astype(int)
        df_enhanced['is_university_segment'] = df_enhanced['segment'].str.contains('UNIVERSITY', na=False).astype(int)
    
    # =============================
    # 7. RISK & STABILITY FEATURES
    # =============================
    print("⚖️ Creating risk and stability features...")
    
    # Customer stability score
    stability_score = 0
    
    if 'seniority' in df_enhanced.columns:
        seniority_stable = df_enhanced['seniority'] >= 12
        stability_score += seniority_stable.fillna(False).astype(int)
    
    if 'age' in df_enhanced.columns:
        age_stable = df_enhanced['age'] >= 30
        stability_score += age_stable.fillna(False).astype(int)
    
    stability_score += df_enhanced['is_primary_customer_flag']
    stability_score += df_enhanced['is_employee']
    
    df_enhanced['customer_stability_score'] = stability_score
    df_enhanced['is_stable_customer'] = (stability_score >= 2).astype(int)
    
    # Potential value score
    potential_score = 0
    
    if 'age' in df_enhanced.columns:
        age_prime = (df_enhanced['age'] >= 25) & (df_enhanced['age'] <= 55)
        potential_score += age_prime.fillna(False).astype(int)
    
    potential_score += df_enhanced['is_digital_channel'] if 'is_digital_channel' in df_enhanced.columns else 0
    potential_score += df_enhanced['is_domestic'] if 'is_domestic' in df_enhanced.columns else 0
    
    df_enhanced['customer_potential_score'] = potential_score
    df_enhanced['is_high_potential'] = (potential_score >= 2).astype(int)
    
    print(f"✅ Feature engineering completed!")
    print(f"Original features: {df.shape[1]}")
    print(f"Enhanced features: {df_enhanced.shape[1]}")
    print(f"New features added: {df_enhanced.shape[1] - df.shape[1]}")
    
    return df_enhanced

# =============================
# APPLY FEATURE ENGINEERING
# =============================
print("🔍 Checking available columns before feature engineering:")
print(f"Available columns: {df.columns.tolist()}")

df_enhanced = create_enhanced_features(df)

# =============================
# VERIFY DATA TYPES
# =============================
print("\n🔍 CHECKING NEW FEATURES DATA TYPES...")
new_features = [col for col in df_enhanced.columns if col not in df.columns]
print(f"New features created: {len(new_features)}")

for feature in new_features[:15]:  # Show first 15 new features
    print(f"   {feature}: {df_enhanced[feature].dtype}")

print(f"\n📊 SAMPLE OF ENHANCED DATA:")
if len(new_features) > 0:
    sample_features = new_features[:5] if len(new_features) >= 5 else new_features
    print(df_enhanced[sample_features].head())
else:
    print("No new features were created.")

🔍 Checking available columns before feature engineering:
Available columns: ['date', 'customer_id', 'employee_index', 'country_of_residence', 'gender', 'age', 'customer_tenure_days', 'seniority', 'residence_index', 'foreigner_index', 'spouse_index', 'channel', 'deceased_index', 'province_name', 'segment', 'was_primary_customer', 'current_accounts_final_label', 'payroll_accounts_final_label', 'junior_accounts_final_label', 'more_particular_accounts_final_label', 'particular_accounts_final_label', 'particular_plus_accounts_final_label', 'home_account_final_label', 'payroll_final_label', 'e_account_final_label']
🔧 CREATING ENHANCED FEATURES...
📋 Available columns: 25
📅 Creating time-based features...
👥 Creating demographic features...
🏦 Creating banking relationship features...
🔗 Creating interaction features...
🎯 Creating behavioral features...
⚖️ Creating risk and stability features...
✅ Feature engineering completed!
Original features: 25
Enhanced features: 58
New features added: 33

🔍

In [17]:
del df

In [18]:
df_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 442736 entries, 3 to 4606305
Data columns (total 58 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   date                                  442736 non-null  datetime64[ns]
 1   customer_id                           442736 non-null  int32         
 2   employee_index                        415002 non-null  object        
 3   country_of_residence                  415002 non-null  object        
 4   gender                                414987 non-null  object        
 5   age                                   415002 non-null  float64       
 6   customer_tenure_days                  415002 non-null  float64       
 7   seniority                             414995 non-null  float64       
 8   residence_index                       415002 non-null  object        
 9   foreigner_index                       415002 non-null  object  

In [19]:
# =============================
# FEATURE LISTS FOR MODELING
# =============================

# Numeric features (engineered features)
numeric_features = [
    'year', 'month', 'quarter', 'day_of_week', 'is_weekend', 'is_month_end', 'is_quarter_end',
    'days_since_registration', 'registration_year', 'registration_month', 'years_since_registration',
    'is_young_adult', 'is_middle_aged', 'is_senior', 'age_squared',
    'income_vs_median', 'is_high_income', 'is_low_income', 'log_income',
    'seniority_years', 'is_new_relationship', 'is_established_relationship',
    'is_primary_customer', 'is_new_customer', 'is_active', 'is_domestic', 'is_foreigner',
    'total_products', 'has_any_product', 'is_single_product', 'is_multi_product',
    'product_diversity_ratio', 'has_current_account', 'has_savings_account', 'has_premium_account',
    'age_income_interaction', 'income_per_age', 'seniority_income_interaction', 'income_growth_proxy',
    'young_high_income', 'senior_established', 'is_digital_channel',
    'is_vip_segment', 'is_university_segment', 'customer_stability_score',
    'is_stable_customer', 'customer_potential_score', 'is_high_potential'
]

# Categorical features
categorical_features = [
    'tenure_category', 'age_group', 'income_quartile', 'seniority_category', 'channel_type'
]

# Target columns
target_cols = [
    'current_accounts_final_label',
    'payroll_accounts_final_label',
    'junior_accounts_final_label',
    'more_particular_accounts_final_label',
    'particular_accounts_final_label',
    'particular_plus_accounts_final_label',
    'home_account_final_label',
    'payroll_final_label',
    'e_account_final_label'
]

print(f"✅ FEATURE LISTS DEFINED:")
print(f"   Numeric features: {len(numeric_features)}")
print(f"   Categorical features: {len(categorical_features)}")
print(f"   Target columns: {len(target_cols)}")
print(f"   Total features: {len(numeric_features) + len(categorical_features)}")


✅ FEATURE LISTS DEFINED:
   Numeric features: 48
   Categorical features: 5
   Target columns: 9
   Total features: 53


In [20]:
# =============================
# PREPARE DATA FOR MODELING
# =============================

# Features to exclude from modeling
exclude_features = [
    'date', 'customer_id', 'registration_date', 'last_primary_date'
]

# Filter features that exist in dataframe
available_num_cols = [col for col in numeric_features if col in df_enhanced.columns and col not in exclude_features]
available_cat_cols = [col for col in categorical_features if col in df_enhanced.columns and col not in exclude_features]

print(f"🎯 FINAL FEATURE SELECTION:")
print(f"Available numeric features: {len(available_num_cols)}")
print(f"Available categorical features: {len(available_cat_cols)}")
print(f"Target labels: {len(target_cols)}")

# Prepare X and y
X = df_enhanced[available_num_cols + available_cat_cols]
y = df_enhanced[target_cols]

print(f"\n📋 DATA SHAPE:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Check for any remaining issues
print(f"\n🔍 DATA QUALITY CHECK:")
print(f"Missing values in X: {X.isnull().sum().sum()}")
print(f"Missing values in y: {y.isnull().sum().sum()}")

🎯 FINAL FEATURE SELECTION:
Available numeric features: 24
Available categorical features: 4
Target labels: 9

📋 DATA SHAPE:
X shape: (442736, 28)
y shape: (442736, 9)

🔍 DATA QUALITY CHECK:
Missing values in X: 169929
Missing values in y: 0


# Test dataset

In [21]:
test_10_2015 = pd.read_csv('data/processed/test_10_2015.csv')
test_11_2015 = pd.read_csv('data/processed/test_11_2015.csv')
test_12_2015 = pd.read_csv('data/processed/test_12_2015.csv')

In [22]:
test_10_2015 = reduce_memory_usage(test_10_2015)
test_11_2015 = reduce_memory_usage(test_11_2015)
test_12_2015 = reduce_memory_usage(test_12_2015)

Mem. usage decreased to 903.96 Mb (53.6% reduction)
Mem. usage decreased to 950.77 Mb (53.6% reduction)
Mem. usage decreased to 998.31 Mb (53.6% reduction)


In [23]:
test_10_2015 = clean_dataset(test_10_2015)
test_11_2015 = clean_dataset(test_11_2015)
test_12_2015 = clean_dataset(test_12_2015)


🧹 CLEANING DATASET...
1️⃣ Handling missing values...
   ✅ Filled payroll_final_label: 0 missing → 0
   ✅ Filled pensions_2_final_label: 0 missing → 0

2️⃣ Creating customer tenure feature...
   ✅ Created 'customer_tenure_days' feature
   📊 Range: -3.0 to 7590.0 days
   🗑️ Dropped 'registration_date' column

3️⃣ Converting last_primary_date to binary indicator...
   ✅ Created 'was_primary_customer' binary feature
   📊 5,316,273 nulls → 0, 8,870 dates → 1
   🗑️ Dropped 'last_primary_date' column

4️⃣ Removing constant and duplicate columns...
   🗑️ Dropped 'address_type' (constant value)
   🗑️ Dropped 'province_code' (duplicate of province_name)

5️⃣ Cleaning numeric columns...
   ✅ Cleaned 'age': object → numeric (9,750 nulls)
   ⚠️ Converted 14 negative seniority values to NaN
   ✅ Cleaned 'seniority': object → numeric (9,764 nulls)

📊 CLEANUP SUMMARY:
----------------------------------------
   Final shape: (5325143, 46)
   Memory usage: 3423.5 MB
   Null values: 6,810,911

📋 DATA TYP

In [24]:
test_10_2015 = filter_data(test_10_2015)
test_11_2015 = filter_data(test_11_2015)
test_12_2015 = filter_data(test_12_2015)


Tổng số khách hàng: 1,017,191
Khách hàng KHÔNG sở hữu tài khoản thanh toán nào: 1,017,191

📊 Distribution check:
  current_accounts_final_label: [0 1]
  payroll_accounts_final_label: [0 1]
  junior_accounts_final_label: [0 1]
  more_particular_accounts_final_label: [0 1]
  particular_accounts_final_label: [0 1]
  particular_plus_accounts_final_label: [0 1]
  home_account_final_label: [0]
  payroll_final_label: [0 1]
  e_account_final_label: [0 1]
Tổng số khách hàng: 1,223,624
Khách hàng KHÔNG sở hữu tài khoản thanh toán nào: 1,223,624

📊 Distribution check:
  current_accounts_final_label: [0 1]
  payroll_accounts_final_label: [0 1]
  junior_accounts_final_label: [0 1]
  more_particular_accounts_final_label: [0 1]
  particular_accounts_final_label: [0 1]
  particular_plus_accounts_final_label: [0 1]
  home_account_final_label: [0]
  payroll_final_label: [0 1]
  e_account_final_label: [0 1]
Tổng số khách hàng: 1,425,848
Khách hàng KHÔNG sở hữu tài khoản thanh toán nào: 1,425,848

📊 Distr

In [25]:
test_10_2015 = create_enhanced_features(test_10_2015)
test_11_2015 = create_enhanced_features(test_11_2015)
test_12_2015 = create_enhanced_features(test_12_2015)

🔧 CREATING ENHANCED FEATURES...
📋 Available columns: 25
📅 Creating time-based features...
👥 Creating demographic features...
🏦 Creating banking relationship features...
🔗 Creating interaction features...
🎯 Creating behavioral features...
⚖️ Creating risk and stability features...
✅ Feature engineering completed!
Original features: 25
Enhanced features: 58
New features added: 33
🔧 CREATING ENHANCED FEATURES...
📋 Available columns: 25
📅 Creating time-based features...
👥 Creating demographic features...
🏦 Creating banking relationship features...
🔗 Creating interaction features...
🎯 Creating behavioral features...
⚖️ Creating risk and stability features...
✅ Feature engineering completed!
Original features: 25
Enhanced features: 58
New features added: 33
🔧 CREATING ENHANCED FEATURES...
📋 Available columns: 25
📅 Creating time-based features...
👥 Creating demographic features...
🏦 Creating banking relationship features...
🔗 Creating interaction features...
🎯 Creating behavioral features...


# Evaluate function

In [26]:
def apk(actual, predicted, k=7):
    """
    Computes the average precision at k.
    
    Parameters:
    actual : list - A list of actual relevant items
    predicted : list - A list of predicted items ordered by rank
    k : int - The maximum number of predicted elements
    
    Returns:
    score : double - The average precision at k
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=7):
    """
    Computes the mean average precision at k.
    
    Parameters:
    actual : list of lists - A list of lists of actual relevant items
    predicted : list of lists - A list of lists of predicted items ordered by rank
    k : int - The maximum number of predicted elements
    
    Returns:
    score : double - The mean average precision at k
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def evaluate_recommendations(actual_products, predicted_products, k=7):
    """
    Evaluate recommendation system using MAP@k
    
    Parameters:
    actual_products : dict - {user_id: [list of actual products]}
    predicted_products : dict - {user_id: [list of predicted products]}
    k : int - Number of recommendations to consider
    
    Returns:
    map_score : float - MAP@k score
    """
    actual_list = []
    predicted_list = []
    
    for user_id in actual_products.keys():
        if user_id in predicted_products:
            actual_list.append(actual_products[user_id])
            predicted_list.append(predicted_products[user_id])
        else:
            actual_list.append(actual_products[user_id])
            predicted_list.append([])  # No predictions for this user
    
    map_score = mapk(actual_list, predicted_list, k)
    
    print(f" MAP@{k}: {map_score:.4f}")
    print(f" Users evaluated: {len(actual_list):,}")
    print(f" Coverage: {len([p for p in predicted_list if p]) / len(predicted_list):.2%}")
    
    return map_score

In [27]:

def precision_at_k(actual, predicted, k=7):
    """
    Computes the precision at k.

    Parameters:
    actual : list - A list of actual relevant items
    predicted : list - A list of predicted items ordered by rank
    k : int - The maximum number of predicted elements

    Returns:
    precision : double - The precision at k
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    
    if not predicted:
        return 0.0
    
    # Đếm số dự đoán đúng trong top-k
    correct = len(set(predicted) & set(actual))
    return correct / k


def evaluate_recommendations_with_precision(actual_products, predicted_products, k=7):
    """
    Evaluate recommendation system using MAP@k and Precision@k.

    Parameters:
    actual_products : dict - {user_id: [list of actual products]}
    predicted_products : dict - {user_id: [list of predicted products]}
    k : int - Number of recommendations to consider

    Returns:
    metrics : dict - {'MAP@k': ..., 'Precision@k': ...}
    """
    actual_list = []
    predicted_list = []

    for user_id in actual_products.keys():
        if user_id in predicted_products:
            actual_list.append(actual_products[user_id])
            predicted_list.append(predicted_products[user_id])
        else:
            actual_list.append(actual_products[user_id])
            predicted_list.append([])  # No predictions for this user

    # MAP@k
    map_score = mapk(actual_list, predicted_list, k)

    # Precision@k trung bình
    precision_scores = [
        precision_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)
    ]
    avg_precision = np.mean(precision_scores)

    print(f" MAP@{k}: {map_score:.4f}")
    print(f" Precision@{k}: {avg_precision:.4f}")
    print(f" Users evaluated: {len(actual_list):,}")
    print(f" Coverage: {len([p for p in predicted_list if p]) / len(predicted_list):.2%}")

    return {f"MAP@{k}": map_score, f"Precision@{k}": avg_precision}

# Data Pipline

In [28]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Updated preprocessors
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),   # Fill missing values with mean
    ('scaler', StandardScaler())                   # Scale numerical data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing with most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))     # One-hot encode categorical data
])

# Updated ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, available_num_cols),
        ('cat', categorical_transformer, available_cat_cols)
    ],
    remainder='drop'  # Drop any remaining columns
)

print(f"✅ Preprocessor updated with:")
print(f"   - {len(available_num_cols)} numeric features")
print(f"   - {len(available_cat_cols)} categorical features")

✅ Preprocessor updated with:
   - 24 numeric features
   - 4 categorical features


In [29]:
# =============================
# MULTI-LABEL CLASSIFICATION MODELS - FIXED
# =============================

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score
import time

# Define all models for multi-label classification
models = {
    'Random Forest': MultiOutputClassifier(
        RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        )
    )
}

# Add XGBoost if available - FIXED with proper parameters
try:
    from xgboost import XGBClassifier
    models['XGBoost'] = MultiOutputClassifier(
        XGBClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42,
            n_jobs=-1,
            verbosity=0,
            base_score=0.5,  # Fix for the logistic loss error
            objective='binary:logistic',  # Explicitly set objective
            eval_metric='logloss'  # Set evaluation metric
        )
    )
    print("✅ XGBoost added to models (with fixed parameters)")
except ImportError:
    print("⚠️ XGBoost not available - install with: pip install xgboost")

# Add LightGBM if available - with proper parameters
try:
    from lightgbm import LGBMClassifier
    models['LightGBM'] = MultiOutputClassifier(
        LGBMClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42,
            n_jobs=-1,
            verbose=-1,
            objective='binary',  # Explicitly set objective
            boosting_type='gbdt'
        )
    )
    print("✅ LightGBM added to models")
except ImportError:
    print("⚠️ LightGBM not available - install with: pip install lightgbm")

# Create complete pipelines for all models
pipelines = {}
for name, model in models.items():
    pipelines[name] = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

print(f"\n🤖 MODELS PREPARED: {list(pipelines.keys())}")
print(f"📊 Total models to train: {len(pipelines)}")

✅ XGBoost added to models (with fixed parameters)
✅ LightGBM added to models

🤖 MODELS PREPARED: ['Random Forest', 'XGBoost', 'LightGBM']
📊 Total models to train: 3


In [30]:
# =============================
# ADDITIONAL METRIC FUNCTIONS - AUC@k AND HIT RATE@k
# =============================

def auc_at_k(actual, predicted_scores, k):
    """
    Calculate AUC@k metric
    
    Parameters:
    actual : list - List of actual relevant items
    predicted_scores : list of tuples - List of (item, score) tuples sorted by score descending
    k : int - Number of top predictions to consider
    
    Returns:
    auc_score : float - AUC@k score
    """
    if not actual:
        return 0.0
    
    # Get top k predictions
    top_k_predictions = predicted_scores[:k]
    
    # Create binary relevance labels for top k
    relevance_labels = []
    scores = []
    
    for item, score in top_k_predictions:
        relevance_labels.append(1 if item in actual else 0)
        scores.append(score)
    
    # If all predictions are relevant or all are irrelevant, return appropriate score
    if len(set(relevance_labels)) == 1:
        return 1.0 if relevance_labels[0] == 1 else 0.0
    
    # Calculate AUC using sklearn
    try:
        from sklearn.metrics import roc_auc_score
        auc_score = roc_auc_score(relevance_labels, scores)
        return auc_score
    except:
        # Fallback manual calculation
        return manual_auc_calculation(relevance_labels, scores)

def manual_auc_calculation(labels, scores):
    """Manual AUC calculation as fallback"""
    # Sort by scores descending
    pairs = list(zip(labels, scores))
    pairs.sort(key=lambda x: x[1], reverse=True)
    
    # Count concordant and discordant pairs
    concordant = 0
    discordant = 0
    
    for i in range(len(pairs)):
        for j in range(i + 1, len(pairs)):
            if pairs[i][0] > pairs[j][0]:  # i is more relevant than j
                if pairs[i][1] > pairs[j][1]:  # i has higher score
                    concordant += 1
                elif pairs[i][1] < pairs[j][1]:  # i has lower score
                    discordant += 1
    
    if concordant + discordant == 0:
        return 0.5
    
    return concordant / (concordant + discordant)

def hit_rate_at_k(actual, predicted, k):
    """
    Calculate Hit Rate@k metric
    
    Parameters:
    actual : list - List of actual relevant items
    predicted : list - List of predicted items (ranked)
    k : int - Number of top predictions to consider
    
    Returns:
    hit_rate : float - Hit Rate@k score (1 if any relevant item in top k, 0 otherwise)
    """
    if not actual:
        return 0.0
    
    # Get top k predictions
    top_k = predicted[:k]
    
    # Check if any actual item is in top k
    for item in actual:
        if item in top_k:
            return 1.0
    
    return 0.0

# =============================
# COMPREHENSIVE MODEL EVALUATION FUNCTION - UPDATED WITH AUC@k AND HIT RATE@k
# =============================

def evaluate_model_comprehensive(model_pipeline, model_name, processed_test_data, tar_cols):
    """
    Comprehensive evaluation function for recommendation system
    
    Parameters:
    model_pipeline : sklearn.pipeline.Pipeline - Trained model pipeline
    model_name : str - Name of the model
    processed_test_data : dict - Dictionary of processed test datasets
    tar_cols : list - List of target column names
    
    Returns:
    results_df : pd.DataFrame - Comprehensive results DataFrame
    detailed_results : dict - Detailed results for further analysis
    """
    
    print(f"\n🧪 EVALUATING {model_name.upper()}...")
    print("="*60)
    
    all_results = []
    detailed_results = {
        'model_name': model_name,
        'monthly_results': {},
        'overall_metrics': {},
        'all_actual_products': {},
        'all_predicted_products': {},
        'all_predicted_scores': {}  # Store scores for AUC calculation
    }
    
    customer_offset = 0
    
    for month_name, data in processed_test_data.items():
        print(f"\n📊 Evaluating on {month_name}...")
        print("-" * 40)
        
        X_test_month = data['X']
        y_test_month = data['y']
        
        if y_test_month is not None:
            # Make predictions
            start_time = time.time()
            y_pred_month = model_pipeline.predict(X_test_month)
            prediction_time = time.time() - start_time
            
            # Get prediction probabilities for ranking
            y_pred_proba = model_pipeline.predict_proba(X_test_month)
            
            # Calculate standard classification metrics
            test_accuracy = accuracy_score(y_test_month, y_pred_month)
            hamming_loss_score = hamming_loss(y_test_month, y_pred_month)
            jaccard_score_macro = jaccard_score(y_test_month, y_pred_month, average='macro')
            
            print(f"   📈 Standard Metrics:")
            print(f"      Accuracy: {test_accuracy:.4f}")
            print(f"      Hamming Loss: {hamming_loss_score:.4f}")
            print(f"      Jaccard (Macro): {jaccard_score_macro:.4f}")
            
            # =============================
            # PREPARE RECOMMENDATION DATA
            # =============================
            actual_products = {}
            predicted_products = {}
            predicted_scores = {}  # Store scores for AUC calculation
            
            for i in range(len(y_test_month)):
                customer_id = i
                
                # Get actual products
                actual = []
                for j, product in enumerate(tar_cols):
                    if y_test_month.iloc[i, j] == 1:
                        actual.append(product)
                actual_products[customer_id] = actual
                
                # Get predicted products (ranked by probability)
                product_scores = []
                for j, product in enumerate(tar_cols):
                    # Get probability of class 1
                    prob = y_pred_proba[j][i][1] if len(y_pred_proba[j][i]) > 1 else y_pred_proba[j][i][0]
                    product_scores.append((product, prob))
                
                # Sort by probability (descending)
                product_scores.sort(key=lambda x: x[1], reverse=True)
                predicted = [product for product, score in product_scores]
                predicted_products[customer_id] = predicted
                predicted_scores[customer_id] = product_scores
                
                # Add to overall data with offset
                detailed_results['all_actual_products'][customer_id + customer_offset] = actual
                detailed_results['all_predicted_products'][customer_id + customer_offset] = predicted
                detailed_results['all_predicted_scores'][customer_id + customer_offset] = product_scores
            
            customer_offset += len(actual_products)
            
            # =============================
            # CALCULATE RECOMMENDATION METRICS - WITH AUC@k AND HIT RATE@k
            # =============================
            print(f"\n   🎯 Recommendation Metrics:")
            
            month_metrics = {}
            for k in [2, 3, 4, 5]:
                # Calculate MAP@k and Precision@k
                actual_list = list(actual_products.values())
                predicted_list = list(predicted_products.values())
                predicted_scores_list = list(predicted_scores.values())
                
                # MAP@k
                map_score = mapk(actual_list, predicted_list, k)
                
                # Precision@k
                precision_scores = [precision_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)]
                avg_precision = np.mean(precision_scores)
                
                # AUC@k
                auc_scores = [auc_at_k(a, ps, k) for a, ps in zip(actual_list, predicted_scores_list)]
                avg_auc = np.mean(auc_scores)
                
                # Hit Rate@k
                hit_rate_scores = [hit_rate_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)]
                avg_hit_rate = np.mean(hit_rate_scores)
                
                month_metrics[f'MAP@{k}'] = map_score
                month_metrics[f'Precision@{k}'] = avg_precision
                month_metrics[f'AUC@{k}'] = avg_auc
                month_metrics[f'HitRate@{k}'] = avg_hit_rate
                
                print(f"      k={k}: MAP={map_score:.4f}, Precision={avg_precision:.4f}, AUC={avg_auc:.4f}, HitRate={avg_hit_rate:.4f}")
            
            # =============================
            # MONTHLY STATISTICS
            # =============================
            total_customers = len(actual_products)
            customers_with_products = len([p for p in actual_products.values() if p])
            avg_products_per_customer = np.mean([len(p) for p in actual_products.values()])
            
            # Product distribution
            product_counts = {}
            for products in actual_products.values():
                for product in products:
                    product_counts[product] = product_counts.get(product, 0) + 1
            
            # Store monthly result
            month_result = {
                'Model': model_name,
                'Month': month_name,
                'Total_Customers': total_customers,
                'Customers_With_Products': customers_with_products,
                'Coverage_Rate': customers_with_products / total_customers,
                'Avg_Products_Per_Customer': avg_products_per_customer,
                'Test_Accuracy': test_accuracy,
                'Hamming_Loss': hamming_loss_score,
                'Jaccard_Macro': jaccard_score_macro,
                'Prediction_Time': prediction_time,
                **month_metrics  # Add all MAP@k, Precision@k, AUC@k, HitRate@k metrics
            }
            
            all_results.append(month_result)
            
            # Store detailed results
            detailed_results['monthly_results'][month_name] = {
                'actual_products': actual_products,
                'predicted_products': predicted_products,
                'predicted_scores': predicted_scores,
                'metrics': month_metrics,
                'stats': month_result,
                'product_distribution': product_counts
            }
            
            print(f"   ⏱️ Time: {prediction_time:.2f}s")
        
        else:
            print(f"   ⚠️ No ground truth available for {month_name}")
    
    # =============================
    # CALCULATE OVERALL METRICS - WITH AUC@k AND HIT RATE@k
    # =============================
    if detailed_results['all_actual_products'] and detailed_results['all_predicted_products']:
        print(f"\n🎯 Overall Performance:")
        
        overall_metrics = {}
        for k in [2, 3, 4, 5]:
            actual_list = list(detailed_results['all_actual_products'].values())
            predicted_list = list(detailed_results['all_predicted_products'].values())
            predicted_scores_list = list(detailed_results['all_predicted_scores'].values())
            
            # Overall MAP@k and Precision@k
            map_score = mapk(actual_list, predicted_list, k)
            precision_scores = [precision_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)]
            avg_precision = np.mean(precision_scores)
            
            # Overall AUC@k
            auc_scores = [auc_at_k(a, ps, k) for a, ps in zip(actual_list, predicted_scores_list)]
            avg_auc = np.mean(auc_scores)
            
            # Overall Hit Rate@k
            hit_rate_scores = [hit_rate_at_k(a, p, k) for a, p in zip(actual_list, predicted_list)]
            avg_hit_rate = np.mean(hit_rate_scores)
            
            overall_metrics[f'Overall_MAP@{k}'] = map_score
            overall_metrics[f'Overall_Precision@{k}'] = avg_precision
            overall_metrics[f'Overall_AUC@{k}'] = avg_auc
            overall_metrics[f'Overall_HitRate@{k}'] = avg_hit_rate
            
            print(f"   Overall k={k}: MAP={map_score:.4f}, Precision={avg_precision:.4f}, AUC={avg_auc:.4f}, HitRate={avg_hit_rate:.4f}")
        
        # Add overall row
        overall_result = {
            'Model': model_name,
            'Month': 'OVERALL',
            'Total_Customers': len(detailed_results['all_actual_products']),
            'Customers_With_Products': len([p for p in detailed_results['all_actual_products'].values() if p]),
            'Coverage_Rate': len([p for p in detailed_results['all_actual_products'].values() if p]) / len(detailed_results['all_actual_products']),
            'Avg_Products_Per_Customer': np.mean([len(p) for p in detailed_results['all_actual_products'].values()]),
            'Test_Accuracy': np.mean([r['Test_Accuracy'] for r in all_results]),
            'Hamming_Loss': np.mean([r['Hamming_Loss'] for r in all_results]),
            'Jaccard_Macro': np.mean([r['Jaccard_Macro'] for r in all_results]),
            'Prediction_Time': sum([r['Prediction_Time'] for r in all_results]),
        }
        
        # Add overall metrics - WITH AUC@k AND HIT RATE@k
        for k in [2, 3, 4, 5]:
            overall_result[f'MAP@{k}'] = overall_metrics[f'Overall_MAP@{k}']
            overall_result[f'Precision@{k}'] = overall_metrics[f'Overall_Precision@{k}']
            overall_result[f'AUC@{k}'] = overall_metrics[f'Overall_AUC@{k}']
            overall_result[f'HitRate@{k}'] = overall_metrics[f'Overall_HitRate@{k}']
        
        all_results.append(overall_result)
        detailed_results['overall_metrics'] = overall_metrics
    
    # Create DataFrame
    results_df = pd.DataFrame(all_results)
    
    return results_df, detailed_results

# =============================
# EXAMPLE USAGE WITH VISUALIZATION
# =============================

def visualize_metrics_comparison(results_df, save_path=None):
    """
    Visualize comparison of different metrics across models and months
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Set up the plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    # Create subplots for different metric types
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    fig.suptitle('Recommendation System Performance Metrics', fontsize=16, fontweight='bold')
    
    # Filter out OVERALL rows for monthly comparison
    monthly_data = results_df[results_df['Month'] != 'OVERALL']
    
    # 1. MAP@k comparison
    ax1 = axes[0, 0]
    map_cols = [col for col in results_df.columns if col.startswith('MAP@')]
    for col in map_cols:
        if col in monthly_data.columns:
            ax1.plot(monthly_data['Month'], monthly_data[col], marker='o', linewidth=2, label=col)
    ax1.set_title('Mean Average Precision @k', fontweight='bold')
    ax1.set_xlabel('Month')
    ax1.set_ylabel('MAP Score')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    plt.setp(ax1.get_xticklabels(), rotation=45)
    
    # 2. AUC@k comparison
    ax2 = axes[0, 1]
    auc_cols = [col for col in results_df.columns if col.startswith('AUC@')]
    for col in auc_cols:
        if col in monthly_data.columns:
            ax2.plot(monthly_data['Month'], monthly_data[col], marker='s', linewidth=2, label=col)
    ax2.set_title('Area Under Curve @k', fontweight='bold')
    ax2.set_xlabel('Month')
    ax2.set_ylabel('AUC Score')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    plt.setp(ax2.get_xticklabels(), rotation=45)
    
    # 3. Hit Rate@k comparison
    ax3 = axes[1, 0]
    hitrate_cols = [col for col in results_df.columns if col.startswith('HitRate@')]
    for col in hitrate_cols:
        if col in monthly_data.columns:
            ax3.plot(monthly_data['Month'], monthly_data[col], marker='^', linewidth=2, label=col)
    ax3.set_title('Hit Rate @k', fontweight='bold')
    ax3.set_xlabel('Month')
    ax3.set_ylabel('Hit Rate')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    plt.setp(ax3.get_xticklabels(), rotation=45)
    
    # 4. Precision@k comparison
    ax4 = axes[1, 1]
    precision_cols = [col for col in results_df.columns if col.startswith('Precision@')]
    for col in precision_cols:
        if col in monthly_data.columns:
            ax4.plot(monthly_data['Month'], monthly_data[col], marker='d', linewidth=2, label=col)
    ax4.set_title('Precision @k', fontweight='bold')
    ax4.set_xlabel('Month')
    ax4.set_ylabel('Precision Score')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    plt.setp(ax4.get_xticklabels(), rotation=45)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

def print_metrics_summary(results_df):
    """
    Print a comprehensive summary of all metrics
    """
    print("\n" + "="*80)
    print("📊 COMPREHENSIVE METRICS SUMMARY")
    print("="*80)
    
    # Overall performance
    overall_row = results_df[results_df['Month'] == 'OVERALL']
    if not overall_row.empty:
        print(f"\n🎯 OVERALL PERFORMANCE:")
        print("-" * 40)
        
        for k in [2, 3, 4, 5]:
            print(f"\nTop-{k} Recommendations:")
            print(f"  MAP@{k}:       {overall_row[f'MAP@{k}'].iloc[0]:.4f}")
            print(f"  Precision@{k}: {overall_row[f'Precision@{k}'].iloc[0]:.4f}")
            print(f"  AUC@{k}:       {overall_row[f'AUC@{k}'].iloc[0]:.4f}")
            print(f"  Hit Rate@{k}:  {overall_row[f'HitRate@{k}'].iloc[0]:.4f}")
    
    # Best performing k
    print(f"\n🏆 BEST PERFORMING K VALUES:")
    print("-" * 40)
    
    for metric_prefix in ['MAP', 'Precision', 'AUC', 'HitRate']:
        metric_cols = [col for col in overall_row.columns if col.startswith(f'{metric_prefix}@')]
        if metric_cols:
            best_col = overall_row[metric_cols].idxmax(axis=1).iloc[0]
            best_value = overall_row[best_col].iloc[0]
            k_value = best_col.split('@')[1]
            print(f"  {metric_prefix}: k={k_value} (score={best_value:.4f})")

In [31]:
# =============================
# PREPARE TEST DATA FORMAT FOR EVALUATION
# =============================

def format_test_data_for_evaluation(test_df, available_num_cols, available_cat_cols, tar_cols):
    """
    Format processed test data để có structure phù hợp với evaluation function
    """
    print(f"📋 Formatting test data: {len(test_df):,} samples")
    
    # Thêm missing features nếu cần
    test_formatted = test_df.copy()
    
    # Add missing numeric features
    for feature in available_num_cols:
        if feature not in test_formatted.columns:
            test_formatted[feature] = 0
            print(f"   ⚠️ Added missing numeric: {feature}")
    
    # Add missing categorical features  
    for feature in available_cat_cols:
        if feature not in test_formatted.columns:
            test_formatted[feature] = 'Unknown'
            print(f"   ⚠️ Added missing categorical: {feature}")
    
    # Prepare X và y
    X_test = test_formatted[available_num_cols + available_cat_cols]
    
    # Check target columns
    if all(col in test_formatted.columns for col in tar_cols):
        y_test = test_formatted[tar_cols]
        print(f"   ✅ Ground truth available: {y_test.shape}")
    else:
        y_test = None
        print(f"   ⚠️ No ground truth available")
    
    return {'X': X_test, 'y': y_test}

# Format test datasets
processed_test_data = {
    'October 2015': format_test_data_for_evaluation(
        test_10_2015, available_num_cols, available_cat_cols, tar_cols
    ),
    'November 2015': format_test_data_for_evaluation(
        test_11_2015, available_num_cols, available_cat_cols, tar_cols
    ),
    'December 2015': format_test_data_for_evaluation(
        test_12_2015, available_num_cols, available_cat_cols, tar_cols
    )
}

print(f"\n✅ TEST DATA FORMATTED FOR EVALUATION!")
for month, data in processed_test_data.items():
    x_shape = data['X'].shape
    y_shape = data['y'].shape if data['y'] is not None else 'None'
    print(f"   {month}: X={x_shape}, y={y_shape}")

📋 Formatting test data: 1,017,191 samples
   ✅ Ground truth available: (1017191, 9)
📋 Formatting test data: 1,223,624 samples
   ✅ Ground truth available: (1223624, 9)
📋 Formatting test data: 1,425,848 samples
   ✅ Ground truth available: (1425848, 9)

✅ TEST DATA FORMATTED FOR EVALUATION!
   October 2015: X=(1017191, 28), y=(1017191, 9)
   November 2015: X=(1223624, 28), y=(1223624, 9)
   December 2015: X=(1425848, 28), y=(1425848, 9)


In [32]:
# =============================
# TRAIN ALL MODELS AND EVALUATE COMPREHENSIVELY
# =============================




X_train = df_enhanced[available_num_cols + available_cat_cols]
y_train = df_enhanced[tar_cols]

# Storage for all results
all_model_results = []
all_detailed_results = {}
training_results = {}

print("🚀 TRAINING AND EVALUATING ALL MODELS...")
print("="*70)

for model_name, pipeline in pipelines.items():
    print(f"\n🤖 TRAINING {model_name.upper()}...")
    print("-" * 50)
    
    # =============================
    # TRAIN MODEL
    # =============================
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Training metrics
    print("📊 Calculating training metrics...")
    y_pred_train = pipeline.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_pred_train)
    
    print(f"   ✅ Training completed in {training_time:.2f}s")
    print(f"   📈 Train Accuracy: {train_accuracy:.4f}")
    
    # Store training results
    training_results[model_name] = {
        'pipeline': pipeline,
        'train_accuracy': train_accuracy,
        'training_time': training_time
    }
    
    # =============================
    # EVALUATE ON TEST SETS
    # =============================
    model_results_df, detailed_results = evaluate_model_comprehensive(
        pipeline, model_name, processed_test_data, tar_cols
    )
    
    # Add training info to results
    model_results_df['Training_Time'] = training_time
    model_results_df['Train_Accuracy'] = train_accuracy
    
    # Store results
    all_model_results.append(model_results_df)
    all_detailed_results[model_name] = detailed_results
    
    print(f"\n✅ {model_name} evaluation completed!")
    print(f"🔧 Pipeline saved for future use")

# =============================
# COMBINE ALL RESULTS
# =============================
print(f"\n🎉 ALL MODELS TRAINED AND EVALUATED!")
final_results_df = pd.concat(all_model_results, ignore_index=True)

print(f"📊 SUMMARY:")
print(f"   Models evaluated: {len(pipelines)}")
print(f"   Total result rows: {len(final_results_df)}")
print(f"   Test datasets: {len(processed_test_data)}")

🚀 TRAINING AND EVALUATING ALL MODELS...

🤖 TRAINING RANDOM FOREST...
--------------------------------------------------
📊 Calculating training metrics...
   ✅ Training completed in 81.68s
   📈 Train Accuracy: 0.9509

🧪 EVALUATING RANDOM FOREST...

📊 Evaluating on October 2015...
----------------------------------------
   📈 Standard Metrics:
      Accuracy: 0.9675
      Hamming Loss: 0.0041
      Jaccard (Macro): 0.0101

   🎯 Recommendation Metrics:
      k=2: MAP=0.0118, Precision=0.0123, AUC=0.0000, HitRate=0.0246
      k=3: MAP=0.0129, Precision=0.0097, AUC=0.0118, HitRate=0.0281
      k=4: MAP=0.0136, Precision=0.0080, AUC=0.0168, HitRate=0.0294
      k=5: MAP=0.0140, Precision=0.0068, AUC=0.0197, HitRate=0.0301
   ⏱️ Time: 11.80s

📊 Evaluating on November 2015...
----------------------------------------
   📈 Standard Metrics:
      Accuracy: 0.9696
      Hamming Loss: 0.0038
      Jaccard (Macro): 0.0102

   🎯 Recommendation Metrics:
      k=2: MAP=0.0108, Precision=0.0112, AUC=0.

In [37]:
# =============================
# COMPREHENSIVE MODEL COMPARISON AND SUMMARY - WITH ALL METRICS
# =============================

# Display final results
print("📊 DETAILED RESULTS TABLE:")
print("="*80)
display(final_results_df)

# =============================
# MODEL PERFORMANCE SUMMARY - ENHANCED WITH ALL METRICS
# =============================

print("\n🏆 MODEL PERFORMANCE SUMMARY:")
print("="*80)

# Overall performance comparison (filter OVERALL rows only)
overall_results = final_results_df[final_results_df['Month'] == 'OVERALL'].copy()

if not overall_results.empty:
    # Sort by MAP@5 (primary metric)
    overall_results_sorted = overall_results.sort_values('MAP@5', ascending=False)
    
    print("\n🎯 RANKING BY MAP@5:")
    print("-" * 50)
    for i, (_, row) in enumerate(overall_results_sorted.iterrows(), 1):
        print(f"{i}. {row['Model']}: MAP@5 = {row['MAP@5']:.4f}")
    
    # Comprehensive metrics comparison table
    print(f"\n📋 COMPREHENSIVE METRICS COMPARISON:")
    comparison_metrics = ['Model', 
                         'MAP@2', 'MAP@3', 'MAP@4', 'MAP@5',
                         'Precision@2', 'Precision@3', 'Precision@4', 'Precision@5',
                         'AUC@2', 'AUC@3', 'AUC@4', 'AUC@5',
                         'HitRate@2', 'HitRate@3', 'HitRate@4', 'HitRate@5',
                         'Test_Accuracy', 'Training_Time', 'Prediction_Time']
    
    # Check which metrics are available
    available_metrics = [metric for metric in comparison_metrics if metric in overall_results.columns]
    summary_table = overall_results[available_metrics].round(4)
    display(summary_table)

# =============================
# DETAILED PERFORMANCE ANALYSIS - ALL METRICS
# =============================

print(f"\n🔍 DETAILED PERFORMANCE ANALYSIS:")
print("=" * 80)

for model_name in overall_results['Model'].values:
    print(f"\n🤖 {model_name.upper()} PERFORMANCE:")
    print("-" * 60)
    
    model_data = overall_results[overall_results['Model'] == model_name].iloc[0]
    
    # MAP@k scores
    print(f"   📈 MAP Scores:")
    for k in [2, 3, 4, 5]:
        if f'MAP@{k}' in model_data:
            map_score = model_data[f'MAP@{k}']
            print(f"      MAP@{k}: {map_score:.4f}")
    
    # Precision@k scores  
    print(f"   🎯 Precision Scores:")
    for k in [2, 3, 4, 5]:
        if f'Precision@{k}' in model_data:
            precision_score = model_data[f'Precision@{k}']
            print(f"      Precision@{k}: {precision_score:.4f}")
    
    # AUC@k scores
    print(f"   📊 AUC Scores:")
    for k in [2, 3, 4, 5]:
        if f'AUC@{k}' in model_data:
            auc_score = model_data[f'AUC@{k}']
            print(f"      AUC@{k}: {auc_score:.4f}")
    
    # Hit Rate@k scores
    print(f"   🎲 Hit Rate Scores:")
    for k in [2, 3, 4, 5]:
        if f'HitRate@{k}' in model_data:
            hitrate_score = model_data[f'HitRate@{k}']
            print(f"      HitRate@{k}: {hitrate_score:.4f}")
    
    # Other metrics
    print(f"   ⚡ Performance Metrics:")
    print(f"      Test Accuracy: {model_data['Test_Accuracy']:.4f}")
    if 'Training_Time' in model_data:
        print(f"      Training Time: {model_data['Training_Time']:.2f}s")
    print(f"      Prediction Time: {model_data['Prediction_Time']:.2f}s")
    print(f"      Total Customers: {model_data['Total_Customers']:,}")

# =============================
# MONTHLY PERFORMANCE BREAKDOWN - ALL METRICS
# =============================

print(f"\n📅 MONTHLY PERFORMANCE BREAKDOWN:")
print("=" * 80)

monthly_results = final_results_df[final_results_df['Month'] != 'OVERALL'].copy()

if not monthly_results.empty:
    # Group by month and show key metrics for each model
    for month in monthly_results['Month'].unique():
        print(f"\n📊 {month}:")
        print("-" * 40)
        month_data = monthly_results[monthly_results['Month'] == month]
        month_sorted = month_data.sort_values('MAP@5', ascending=False)
        
        for _, row in month_sorted.iterrows():
            print(f"   {row['Model']}:")
            print(f"      MAP@5={row['MAP@5']:.4f}, Precision@5={row['Precision@5']:.4f}")
            if 'AUC@5' in row:
                print(f"      AUC@5={row['AUC@5']:.4f}, HitRate@5={row['HitRate@5']:.4f}")
            print(f"      Accuracy={row['Test_Accuracy']:.4f}")
            print()

# =============================
# BEST MODEL IDENTIFICATION - ALL METRICS
# =============================

print(f"\n🏅 BEST MODEL IDENTIFICATION:")
print("=" * 80)

if not overall_results.empty:
    # Best model by different metrics
    best_models = {}
    
    metrics_to_check = []
    # Add MAP metrics
    for k in [2, 3, 4, 5]:
        if f'MAP@{k}' in overall_results.columns:
            metrics_to_check.append(f'MAP@{k}')
    
    # Add Precision metrics
    for k in [2, 3, 4, 5]:
        if f'Precision@{k}' in overall_results.columns:
            metrics_to_check.append(f'Precision@{k}')
    
    # Add AUC metrics
    for k in [2, 3, 4, 5]:
        if f'AUC@{k}' in overall_results.columns:
            metrics_to_check.append(f'AUC@{k}')
    
    # Add Hit Rate metrics
    for k in [2, 3, 4, 5]:
        if f'HitRate@{k}' in overall_results.columns:
            metrics_to_check.append(f'HitRate@{k}')
    
    # Add other metrics
    if 'Test_Accuracy' in overall_results.columns:
        metrics_to_check.append('Test_Accuracy')
    
    print(f"🏆 BEST PERFORMANCE BY METRIC:")
    print("-" * 50)
    
    for metric in metrics_to_check:
        best_idx = overall_results[metric].idxmax()
        best_model = overall_results.loc[best_idx, 'Model']
        best_score = overall_results.loc[best_idx, metric]
        best_models[metric] = (best_model, best_score)
        print(f"   {metric}: {best_model} ({best_score:.4f})")
    
    # Overall winner (most wins)
    model_wins = {}
    for metric, (model, score) in best_models.items():
        model_wins[model] = model_wins.get(model, 0) + 1
    
    overall_winner = max(model_wins.items(), key=lambda x: x[1])
    print(f"\n🎖️ OVERALL WINNER: {overall_winner[0]} ({overall_winner[1]} best scores out of {len(best_models)})")
    
    # Show wins breakdown
    print(f"\n📊 WINS BREAKDOWN:")
    print("-" * 30)
    sorted_wins = sorted(model_wins.items(), key=lambda x: x[1], reverse=True)
    for model, wins in sorted_wins:
        win_percentage = (wins / len(best_models)) * 100
        print(f"   {model}: {wins} wins ({win_percentage:.1f}%)")

# =============================
# PERFORMANCE INSIGHTS - ENHANCED
# =============================

print(f"\n💡 PERFORMANCE INSIGHTS:")
print("=" * 80)

if not overall_results.empty:
    # MAP@k trend analysis
    print(f"📈 MAP@k Trends:")
    print("-" * 30)
    for _, row in overall_results.iterrows():
        model = row['Model']
        map_scores = [row[f'MAP@{k}'] for k in [2, 3, 4, 5] if f'MAP@{k}' in row]
        if len(map_scores) >= 2:
            trend = "📈 Improving" if all(map_scores[i] <= map_scores[i+1] for i in range(len(map_scores)-1)) \
                    else "📉 Declining" if all(map_scores[i] >= map_scores[i+1] for i in range(len(map_scores)-1)) \
                    else "📊 Mixed"
            print(f"   {model}: {trend} (MAP@2={map_scores[0]:.3f} → MAP@5={map_scores[-1]:.3f})")
    
    # AUC@k trend analysis
    if any(f'AUC@{k}' in overall_results.columns for k in [2, 3, 4, 5]):
        print(f"\n📊 AUC@k Trends:")
        print("-" * 30)
        for _, row in overall_results.iterrows():
            model = row['Model']
            auc_scores = [row[f'AUC@{k}'] for k in [2, 3, 4, 5] if f'AUC@{k}' in row]
            if len(auc_scores) >= 2:
                trend = "📈 Improving" if all(auc_scores[i] <= auc_scores[i+1] for i in range(len(auc_scores)-1)) \
                        else "📉 Declining" if all(auc_scores[i] >= auc_scores[i+1] for i in range(len(auc_scores)-1)) \
                        else "📊 Mixed"
                print(f"   {model}: {trend} (AUC@2={auc_scores[0]:.3f} → AUC@5={auc_scores[-1]:.3f})")
    
    # Hit Rate@k trend analysis
    if any(f'HitRate@{k}' in overall_results.columns for k in [2, 3, 4, 5]):
        print(f"\n🎲 Hit Rate@k Trends:")
        print("-" * 30)
        for _, row in overall_results.iterrows():
            model = row['Model']
            hitrate_scores = [row[f'HitRate@{k}'] for k in [2, 3, 4, 5] if f'HitRate@{k}' in row]
            if len(hitrate_scores) >= 2:
                trend = "📈 Improving" if all(hitrate_scores[i] <= hitrate_scores[i+1] for i in range(len(hitrate_scores)-1)) \
                        else "📉 Declining" if all(hitrate_scores[i] >= hitrate_scores[i+1] for i in range(len(hitrate_scores)-1)) \
                        else "📊 Mixed"
                print(f"   {model}: {trend} (HitRate@2={hitrate_scores[0]:.3f} → HitRate@5={hitrate_scores[-1]:.3f})")
    
    # Speed vs Accuracy analysis
    print(f"\n⚡ Speed vs Performance Analysis:")
    print("-" * 40)
    for _, row in overall_results.iterrows():
        model = row['Model']
        prediction_time = row['Prediction_Time']
        map5_score = row['MAP@5']
        
        # Calculate efficiency metrics
        efficiency_ratio = map5_score / prediction_time * 1000 if prediction_time > 0 else 0
        
        if 'Training_Time' in row:
            training_time = row['Training_Time']
            total_efficiency = map5_score / (training_time + prediction_time) * 1000 if (training_time + prediction_time) > 0 else 0
            print(f"   {model}:")
            print(f"      Prediction Efficiency: {efficiency_ratio:.2f} (MAP@5 per ms)")
            print(f"      Total Efficiency: {total_efficiency:.2f} (MAP@5 per ms total)")
        else:
            print(f"   {model}: Prediction Efficiency = {efficiency_ratio:.2f} (MAP@5 per ms)")

# =============================
# METRIC CORRELATION ANALYSIS
# =============================

print(f"\n🔗 METRIC CORRELATION ANALYSIS:")
print("=" * 80)

if not overall_results.empty and len(overall_results) > 1:
    import numpy as np
    
    # Get all numerical metric columns
    metric_columns = []
    for k in [2, 3, 4, 5]:
        for metric_type in ['MAP', 'Precision', 'AUC', 'HitRate']:
            col_name = f'{metric_type}@{k}'
            if col_name in overall_results.columns:
                metric_columns.append(col_name)
    
    if 'Test_Accuracy' in overall_results.columns:
        metric_columns.append('Test_Accuracy')
    
    if len(metric_columns) >= 2:
        print(f"📊 High Correlations (>0.8) between metrics:")
        print("-" * 50)
        
        # Calculate correlation matrix
        corr_matrix = overall_results[metric_columns].corr()
        
        # Find high correlations
        high_correlations = []
        for i in range(len(metric_columns)):
            for j in range(i+1, len(metric_columns)):
                corr_value = corr_matrix.iloc[i, j]
                if abs(corr_value) > 0.8:
                    high_correlations.append((metric_columns[i], metric_columns[j], corr_value))
        
        if high_correlations:
            for metric1, metric2, corr in high_correlations:
                print(f"   {metric1} ↔ {metric2}: {corr:.3f}")
        else:
            print("   No high correlations found (all correlations < 0.8)")

# =============================
# RECOMMENDATION SUMMARY
# =============================

print(f"\n🎁 RECOMMENDATION SUMMARY:")
print("=" * 80)

if not overall_results.empty:
    # Best overall model
    best_map5_model = overall_results.loc[overall_results['MAP@5'].idxmax(), 'Model']
    best_map5_score = overall_results['MAP@5'].max()
    
    print(f"🌟 RECOMMENDED MODEL: {best_map5_model}")
    print(f"   Primary Reason: Best MAP@5 score ({best_map5_score:.4f})")
    
    # Additional insights
    best_model_data = overall_results[overall_results['Model'] == best_map5_model].iloc[0]
    
    print(f"\n📋 Key Performance Metrics for {best_map5_model}:")
    print("-" * 50)
    
    # Show all available metrics for the best model
    for k in [2, 3, 4, 5]:
        metrics_line = []
        if f'MAP@{k}' in best_model_data:
            metrics_line.append(f"MAP@{k}={best_model_data[f'MAP@{k}']:.4f}")
        if f'Precision@{k}' in best_model_data:
            metrics_line.append(f"Precision@{k}={best_model_data[f'Precision@{k}']:.4f}")
        if f'AUC@{k}' in best_model_data:
            metrics_line.append(f"AUC@{k}={best_model_data[f'AUC@{k}']:.4f}")
        if f'HitRate@{k}' in best_model_data:
            metrics_line.append(f"HitRate@{k}={best_model_data[f'HitRate@{k}']:.4f}")
        
        if metrics_line:
            print(f"   k={k}: {', '.join(metrics_line)}")
    
    print(f"\n⚡ Performance: Accuracy={best_model_data['Test_Accuracy']:.4f}, "
          f"Prediction Time={best_model_data['Prediction_Time']:.2f}s")
    
    # Business impact estimation
    total_customers = int(best_model_data['Total_Customers'])
    if 'HitRate@5' in best_model_data:
        hit_rate = best_model_data['HitRate@5']
        estimated_customers_with_good_recommendations = int(total_customers * hit_rate)
        print(f"\n💼 Business Impact Estimate:")
        print(f"   Expected customers with good top-5 recommendations: {estimated_customers_with_good_recommendations:,} "
              f"({hit_rate*100:.1f}% of {total_customers:,})")

print(f"\n" + "="*80)
print("✅ COMPREHENSIVE ANALYSIS COMPLETE!")
print("="*80)

📊 DETAILED RESULTS TABLE:


,Model,Month,Total_Customers,Customers_With_Products,Coverage_Rate,Avg_Products_Per_Customer,Test_Accuracy,Hamming_Loss,Jaccard_Macro,Prediction_Time,...,MAP@4,Precision@4,AUC@4,HitRate@4,MAP@5,Precision@5,AUC@5,HitRate@5,Training_Time,Train_Accuracy
0,Random Forest,October 2015,1017191,31170,0.030643,0.035039,0.967542,0.004093,0.010136,11.795192,...,0.013595,0.007980,0.016758,0.029385,0.013956,0.006753,0.019720,0.030114,81.67847,0.950896
1,Random Forest,November 2015,1223624,34541,0.028228,0.031845,0.969619,0.003778,0.010163,13.866546,...,0.012426,0.007224,0.015498,0.027032,0.012762,0.006121,0.018161,0.027705,81.67847,0.950896
2,Random Forest,December 2015,1425848,31935,0.022397,0.024978,0.975345,0.003025,0.009715,16.190880,...,0.010012,0.005705,0.012581,0.021348,0.010258,0.004812,0.014649,0.021943,81.67847,0.950896
3,Random Forest,OVERALL,3666663,97646,0.026631,0.030061,0.970835,0.003632,0.010005,41.852618,...,0.011811,0.006843,0.014713,0.025474,0.012119,0.005787,0.017228,0.026133,81.67847,0.950896
4,XGBoost,October 2015,1017191,31170,0.030643,0.035039,0.968173,0.004024,0.010722,2.449513,...,0.026565,0.008429,0.026566,0.030007,0.026796,0.006975,0.027307,0.030572,8.86970,0.951054
5,XGBoost,November 2015,1223624,34541,0.028228,0.031845,0.970274,0.003706,0.011692,3.454211,...,0.024351,0.007650,0.024453,0.027668,0.024562,0.006330,0.025108,0.028137,8.86970,0.951054
6,XGBoost,December 2015,1425848,31935,0.022397,0.024978,0.975998,0.002953,0.011390,4.039769,...,0.019719,0.006013,0.019737,0.021895,0.019865,0.004958,0.020194,0.022293,8.86970,0.951054
7,XGBoost,OVERALL,3666663,97646,0.026631,0.030061,0.971482,0.003561,0.011268,9.943493,...,0.023164,0.007229,0.023205,0.026072,0.023355,0.005975,0.023807,0.026540,8.86970,0.951054
8,LightGBM,October 2015,1017191,31170,0.030643,0.035039,0.923488,0.009106,0.005627,4.658453,...,0.025747,0.008376,0.025869,0.029910,0.025941,0.006896,0.026771,0.030306,7.04811,0.939831
9,LightGBM,November 2015,1223624,34541,0.028228,0.031845,0.916547,0.009815,0.004431,5.838314,...,0.023432,0.007581,0.023710,0.027503,0.023617,0.006250,0.024526,0.027876,7.04811,0.939831



🏆 MODEL PERFORMANCE SUMMARY:

🎯 RANKING BY MAP@5:
--------------------------------------------------
1. XGBoost: MAP@5 = 0.0234
2. LightGBM: MAP@5 = 0.0225
3. Random Forest: MAP@5 = 0.0121

📋 COMPREHENSIVE METRICS COMPARISON:


,Model,MAP@2,MAP@3,MAP@4,MAP@5,Precision@2,Precision@3,Precision@4,Precision@5,AUC@2,AUC@3,AUC@4,AUC@5,HitRate@2,HitRate@3,HitRate@4,HitRate@5,Test_Accuracy,Training_Time,Prediction_Time
3,Random Forest,0.0103,0.0113,0.0118,0.0121,0.0108,0.0083,0.0068,0.0058,0.0000,0.0105,0.0147,0.0172,0.0216,0.0244,0.0255,0.0261,0.9708,81.6785,41.8526
7,XGBoost,0.0222,0.0228,0.0232,0.0234,0.0125,0.0091,0.0072,0.0060,0.0216,0.0225,0.0232,0.0238,0.0244,0.0254,0.0261,0.0265,0.9715,8.8697,9.9435
11,LightGBM,0.0213,0.0219,0.0223,0.0225,0.0123,0.0090,0.0072,0.0059,0.0202,0.0216,0.0225,0.0233,0.0239,0.0252,0.0259,0.0263,0.9229,7.0481,17.3415



🔍 DETAILED PERFORMANCE ANALYSIS:

🤖 RANDOM FOREST PERFORMANCE:
------------------------------------------------------------
   📈 MAP Scores:
      MAP@2: 0.0103
      MAP@3: 0.0113
      MAP@4: 0.0118
      MAP@5: 0.0121
   🎯 Precision Scores:
      Precision@2: 0.0108
      Precision@3: 0.0083
      Precision@4: 0.0068
      Precision@5: 0.0058
   📊 AUC Scores:
      AUC@2: 0.0000
      AUC@3: 0.0105
      AUC@4: 0.0147
      AUC@5: 0.0172
   🎲 Hit Rate Scores:
      HitRate@2: 0.0216
      HitRate@3: 0.0244
      HitRate@4: 0.0255
      HitRate@5: 0.0261
   ⚡ Performance Metrics:
      Test Accuracy: 0.9708
      Training Time: 81.68s
      Prediction Time: 41.85s
      Total Customers: 3,666,663

🤖 XGBOOST PERFORMANCE:
------------------------------------------------------------
   📈 MAP Scores:
      MAP@2: 0.0222
      MAP@3: 0.0228
      MAP@4: 0.0232
      MAP@5: 0.0234
   🎯 Precision Scores:
      Precision@2: 0.0125
      Precision@3: 0.0091
      Precision@4: 0.0072
      Pr

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

def visualize_metrics_comprehensive(results_df, save_path=None):
    """
    Comprehensive visualization of all recommendation metrics
    
    Parameters:
    results_df : pd.DataFrame - Results dataframe with all metrics
    save_path : str - Path to save the visualization
    """
    
    # Set up the plotting style
    plt.style.use('default')
    sns.set_palette("husl")
    
    # Filter data
    monthly_data = results_df[results_df['Month'] != 'OVERALL'].copy()
    overall_data = results_df[results_df['Month'] == 'OVERALL'].copy()
    
    if monthly_data.empty:
        print("No monthly data available for visualization")
        return
    
    # Create comprehensive dashboard
    fig = plt.figure(figsize=(24, 20))
    gs = GridSpec(4, 3, figure=fig, hspace=0.3, wspace=0.3)
    
    fig.suptitle('📊 Comprehensive Recommendation System Performance Dashboard', 
                 fontsize=20, fontweight='bold', y=0.98)
    
    # =============================
    # 1. MAP@k COMPARISON (Top Left)
    # =============================
    ax1 = fig.add_subplot(gs[0, 0])
    map_cols = [col for col in monthly_data.columns if col.startswith('MAP@')]
    
    if map_cols:
        for col in map_cols:
            for model in monthly_data['Model'].unique():
                model_data = monthly_data[monthly_data['Model'] == model]
                ax1.plot(model_data['Month'], model_data[col], 
                        marker='o', linewidth=2, markersize=6,
                        label=f"{model} {col}", alpha=0.8)
        
        ax1.set_title('📈 Mean Average Precision @k', fontweight='bold', fontsize=14)
        ax1.set_xlabel('Month', fontweight='bold')
        ax1.set_ylabel('MAP Score', fontweight='bold')
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        ax1.grid(True, alpha=0.3)
        plt.setp(ax1.get_xticklabels(), rotation=45)
        ax1.set_ylim(0, max(monthly_data[map_cols].max()) * 1.1)
    
    # =============================
    # 2. AUC@k COMPARISON (Top Middle)
    # =============================
    ax2 = fig.add_subplot(gs[0, 1])
    auc_cols = [col for col in monthly_data.columns if col.startswith('AUC@')]
    
    if auc_cols:
        for col in auc_cols:
            for model in monthly_data['Model'].unique():
                model_data = monthly_data[monthly_data['Model'] == model]
                ax2.plot(model_data['Month'], model_data[col], 
                        marker='s', linewidth=2, markersize=6,
                        label=f"{model} {col}", alpha=0.8)
        
        ax2.set_title('📊 Area Under Curve @k', fontweight='bold', fontsize=14)
        ax2.set_xlabel('Month', fontweight='bold')
        ax2.set_ylabel('AUC Score', fontweight='bold')
        ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        ax2.grid(True, alpha=0.3)
        plt.setp(ax2.get_xticklabels(), rotation=45)
        ax2.set_ylim(0, 1)
    else:
        ax2.text(0.5, 0.5, 'AUC@k metrics\nnot available', 
                ha='center', va='center', transform=ax2.transAxes, fontsize=12)
        ax2.set_title('📊 Area Under Curve @k', fontweight='bold', fontsize=14)
    
    # =============================
    # 3. HIT RATE@k COMPARISON (Top Right)
    # =============================
    ax3 = fig.add_subplot(gs[0, 2])
    hitrate_cols = [col for col in monthly_data.columns if col.startswith('HitRate@')]
    
    if hitrate_cols:
        for col in hitrate_cols:
            for model in monthly_data['Model'].unique():
                model_data = monthly_data[monthly_data['Model'] == model]
                ax3.plot(model_data['Month'], model_data[col], 
                        marker='^', linewidth=2, markersize=6,
                        label=f"{model} {col}", alpha=0.8)
        
        ax3.set_title('🎲 Hit Rate @k', fontweight='bold', fontsize=14)
        ax3.set_xlabel('Month', fontweight='bold')
        ax3.set_ylabel('Hit Rate', fontweight='bold')
        ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        ax3.grid(True, alpha=0.3)
        plt.setp(ax3.get_xticklabels(), rotation=45)
        ax3.set_ylim(0, 1)
    else:
        ax3.text(0.5, 0.5, 'Hit Rate@k metrics\nnot available', 
                ha='center', va='center', transform=ax3.transAxes, fontsize=12)
        ax3.set_title('🎲 Hit Rate @k', fontweight='bold', fontsize=14)
    
    # =============================
    # 4. PRECISION@k COMPARISON (Second Row Left)
    # =============================
    ax4 = fig.add_subplot(gs[1, 0])
    precision_cols = [col for col in monthly_data.columns if col.startswith('Precision@')]
    
    if precision_cols:
        for col in precision_cols:
            for model in monthly_data['Model'].unique():
                model_data = monthly_data[monthly_data['Model'] == model]
                ax4.plot(model_data['Month'], model_data[col], 
                        marker='d', linewidth=2, markersize=6,
                        label=f"{model} {col}", alpha=0.8)
        
        ax4.set_title('🎯 Precision @k', fontweight='bold', fontsize=14)
        ax4.set_xlabel('Month', fontweight='bold')
        ax4.set_ylabel('Precision Score', fontweight='bold')
        ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        ax4.grid(True, alpha=0.3)
        plt.setp(ax4.get_xticklabels(), rotation=45)
        ax4.set_ylim(0, max(monthly_data[precision_cols].max()) * 1.1)
    
    # =============================
    # 5. OVERALL PERFORMANCE COMPARISON (Second Row Middle)
    # =============================
    ax5 = fig.add_subplot(gs[1, 1])
    
    if not overall_data.empty:
        models = overall_data['Model'].values
        x_pos = np.arange(len(models))
        
        # Create grouped bar chart for key metrics
        width = 0.15
        metrics_to_plot = []
        
        # Get available metrics
        for metric in ['MAP@5', 'Precision@5', 'AUC@5', 'HitRate@5']:
            if metric in overall_data.columns:
                metrics_to_plot.append(metric)
        
        colors = plt.cm.Set3(np.linspace(0, 1, len(metrics_to_plot)))
        
        for i, metric in enumerate(metrics_to_plot):
            values = overall_data[metric].values
            ax5.bar(x_pos + i*width, values, width, 
                   label=metric, alpha=0.8, color=colors[i])
        
        ax5.set_title('🏆 Overall Performance @k=5', fontweight='bold', fontsize=14)
        ax5.set_xlabel('Models', fontweight='bold')
        ax5.set_ylabel('Score', fontweight='bold')
        ax5.set_xticks(x_pos + width * (len(metrics_to_plot)-1) / 2)
        ax5.set_xticklabels(models, rotation=45)
        ax5.legend()
        ax5.grid(True, alpha=0.3, axis='y')
        ax5.set_ylim(0, 1)
    
    # =============================
    # 6. PERFORMANCE vs SPEED ANALYSIS (Second Row Right)
    # =============================
    ax6 = fig.add_subplot(gs[1, 2])
    
    if not overall_data.empty and 'MAP@5' in overall_data.columns:
        x_data = overall_data['Prediction_Time']
        y_data = overall_data['MAP@5']
        models = overall_data['Model']
        
        scatter = ax6.scatter(x_data, y_data, s=100, alpha=0.7, c=range(len(models)), cmap='viridis')
        
        # Add model labels
        for i, model in enumerate(models):
            ax6.annotate(model, (x_data.iloc[i], y_data.iloc[i]), 
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
        
        ax6.set_title('⚡ Performance vs Speed', fontweight='bold', fontsize=14)
        ax6.set_xlabel('Prediction Time (seconds)', fontweight='bold')
        ax6.set_ylabel('MAP@5 Score', fontweight='bold')
        ax6.grid(True, alpha=0.3)
    
    # =============================
    # 7. METRICS CORRELATION HEATMAP (Third Row Left)
    # =============================
    ax7 = fig.add_subplot(gs[2, 0])
    
    if not overall_data.empty:
        # Get numerical columns for correlation
        numerical_cols = []
        for k in [2, 3, 4, 5]:
            for metric_type in ['MAP', 'Precision', 'AUC', 'HitRate']:
                col_name = f'{metric_type}@{k}'
                if col_name in overall_data.columns:
                    numerical_cols.append(col_name)
        
        if len(numerical_cols) >= 2:
            corr_matrix = overall_data[numerical_cols].corr()
            
            # Create heatmap
            sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, 
                       square=True, ax=ax7, cbar_kws={'shrink': 0.8},
                       fmt='.2f', annot_kws={'size': 8})
            
            ax7.set_title('🔗 Metrics Correlation Matrix', fontweight='bold', fontsize=14)
            plt.setp(ax7.get_xticklabels(), rotation=45)
            plt.setp(ax7.get_yticklabels(), rotation=0)
    
    # =============================
    # 8. K-VALUE PERFORMANCE ANALYSIS (Third Row Middle)
    # =============================
    ax8 = fig.add_subplot(gs[2, 1])
    
    if not overall_data.empty:
        # Analyze how metrics change with k
        k_values = [2, 3, 4, 5]
        
        for model in overall_data['Model'].values:
            model_data = overall_data[overall_data['Model'] == model].iloc[0]
            
            # MAP@k progression
            map_scores = []
            for k in k_values:
                if f'MAP@{k}' in model_data:
                    map_scores.append(model_data[f'MAP@{k}'])
                else:
                    map_scores.append(np.nan)
            
            if not all(np.isnan(map_scores)):
                ax8.plot(k_values, map_scores, marker='o', linewidth=2, 
                        label=f'{model} MAP@k', alpha=0.8)
        
        ax8.set_title('📈 MAP@k Progression', fontweight='bold', fontsize=14)
        ax8.set_xlabel('k Value', fontweight='bold')
        ax8.set_ylabel('MAP Score', fontweight='bold')
        ax8.set_xticks(k_values)
        ax8.legend()
        ax8.grid(True, alpha=0.3)
    
    # =============================
    # 9. BUSINESS IMPACT VISUALIZATION (Third Row Right)
    # =============================
    ax9 = fig.add_subplot(gs[2, 2])
    
    if not overall_data.empty and 'HitRate@5' in overall_data.columns:
        models = overall_data['Model'].values
        hit_rates = overall_data['HitRate@5'].values
        total_customers = overall_data['Total_Customers'].iloc[0] if 'Total_Customers' in overall_data.columns else 1000
        
        # Calculate estimated customers with good recommendations
        estimated_customers = hit_rates * total_customers
        
        # Create bar chart
        bars = ax9.bar(range(len(models)), estimated_customers, alpha=0.8, color='lightblue')
        
        # Add percentage labels on bars
        for i, (bar, hit_rate) in enumerate(zip(bars, hit_rates)):
            height = bar.get_height()
            ax9.text(bar.get_x() + bar.get_width()/2., height + total_customers*0.01,
                    f'{hit_rate:.1%}', ha='center', va='bottom', fontweight='bold')
        
        ax9.set_title('💼 Business Impact (Top-5 Recommendations)', fontweight='bold', fontsize=14)
        ax9.set_xlabel('Models', fontweight='bold')
        ax9.set_ylabel('Customers with Good Recommendations', fontweight='bold')
        ax9.set_xticks(range(len(models)))
        ax9.set_xticklabels(models, rotation=45)
        ax9.grid(True, alpha=0.3, axis='y')
    
    # =============================
    # 10. MONTHLY TREND SUMMARY (Bottom Row - Full Width)
    # =============================
    ax10 = fig.add_subplot(gs[3, :])
    
    if not monthly_data.empty:
        # Create summary trend chart
        months = monthly_data['Month'].unique()
        models = monthly_data['Model'].unique()
        
        x_pos = np.arange(len(months))
        width = 0.8 / len(models)
        
        # Use MAP@5 as primary metric
        if 'MAP@5' in monthly_data.columns:
            for i, model in enumerate(models):
                model_monthly = []
                for month in months:
                    month_data = monthly_data[(monthly_data['Month'] == month) & 
                                            (monthly_data['Model'] == model)]
                    if not month_data.empty:
                        model_monthly.append(month_data['MAP@5'].iloc[0])
                    else:
                        model_monthly.append(0)
                
                ax10.bar(x_pos + i*width, model_monthly, width, 
                        label=model, alpha=0.8)
            
            ax10.set_title('📅 Monthly Performance Comparison (MAP@5)', fontweight='bold', fontsize=16)
            ax10.set_xlabel('Month', fontweight='bold')
            ax10.set_ylabel('MAP@5 Score', fontweight='bold')
            ax10.set_xticks(x_pos + width * (len(models)-1) / 2)
            ax10.set_xticklabels(months, rotation=45)
            ax10.legend()
            ax10.grid(True, alpha=0.3, axis='y')
    
    # Save if path provided
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"📊 Visualization saved to: {save_path}")
    
    plt.tight_layout()
    plt.show()

def create_metrics_summary_table(results_df):
    """
    Create a comprehensive summary table of all metrics
    """
    
    overall_data = results_df[results_df['Month'] == 'OVERALL'].copy()
    
    if overall_data.empty:
        print("No overall data available for summary table")
        return None
    
    # Create summary table
    summary_data = []
    
    for _, row in overall_data.iterrows():
        model_summary = {'Model': row['Model']}
        
        # Add all available metrics
        for k in [2, 3, 4, 5]:
            for metric_type in ['MAP', 'Precision', 'AUC', 'HitRate']:
                col_name = f'{metric_type}@{k}'
                if col_name in row:
                    model_summary[col_name] = row[col_name]
        
        # Add other metrics
        if 'Test_Accuracy' in row:
            model_summary['Accuracy'] = row['Test_Accuracy']
        if 'Prediction_Time' in row:
            model_summary['Pred_Time'] = row['Prediction_Time']
        
        summary_data.append(model_summary)
    
    summary_df = pd.DataFrame(summary_data)
    
    # Style the table
    print("\n" + "="*100)
    print("📊 COMPREHENSIVE METRICS SUMMARY TABLE")
    print("="*100)
    
    # Display with proper formatting
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.float_format', '{:.4f}'.format)
    
    print(summary_df.to_string(index=False))
    
    return summary_df



demo_visualization()

🚀 COMPREHENSIVE METRICS VISUALIZATION DEMO

    Usage:

    # Visualize all metrics
    visualize_metrics_comprehensive(
        results_df=final_results_df,
        save_path='comprehensive_metrics_dashboard.png'
    )

    # Create summary table
    summary_table = create_metrics_summary_table(final_results_df)

    Features:
    📈 MAP@k trends across months and models
    📊 AUC@k performance comparison  
    🎲 Hit Rate@k analysis
    🎯 Precision@k evaluation
    🏆 Overall performance comparison
    ⚡ Speed vs accuracy analysis
    🔗 Metrics correlation heatmap
    📅 Monthly trend analysis
    💼 Business impact estimation
    📊 Comprehensive summary table
    


In [38]:
from pathlib import Path
from datetime import datetime
import json
import joblib
import sklearn

# =============================
# SAVE TRAINED PIPELINES
# =============================

MODELS_DIR = Path("models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

def save_pipeline(pipeline, model_name, y_columns=None, extra_meta=None):
    """Save a fitted sklearn Pipeline and a metadata JSON next to it."""
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    safe_name = model_name.replace(" ", "_").lower()
    stem = f"{safe_name}_{ts}"
    model_path = MODELS_DIR / f"{stem}.joblib"
    meta_path = MODELS_DIR / f"{stem}.json"

    joblib.dump(pipeline, model_path)

    meta = {
        "model_name": model_name,
        "created_at": ts,
        "sklearn_version": sklearn.__version__,
        "targets": list(y_columns) if y_columns is not None else None,
    }
    if extra_meta:
        meta.update(extra_meta)

    meta_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2))

    print(f"✅ Saved '{model_name}' to {model_path}")
    print(f"ℹ️  Metadata         → {meta_path}")
    return str(model_path), str(meta_path)

# Save all trained pipelines (from training_results)
saved_artifacts = {}
for model_name, info in training_results.items():
    pipeline_fitted = info['pipeline']
    train_acc = float(info.get('train_accuracy', 0.0))
    train_time = float(info.get('training_time', 0.0))

    model_path, meta_path = save_pipeline(
        pipeline_fitted,
        model_name,
        y_columns=y_train.columns,
        extra_meta={
            "train_accuracy": train_acc,
            "training_time_s": train_time
        }
    )
    saved_artifacts[model_name] = {"model": model_path, "meta": meta_path}

print("\n📦 Saved models summary:")
for k, v in saved_artifacts.items():
    print(f" - {k}: {v['model']}")


✅ Saved 'Random Forest' to models\random_forest_20250820-101435.joblib
ℹ️  Metadata         → models\random_forest_20250820-101435.json
✅ Saved 'XGBoost' to models\xgboost_20250820-101436.joblib
ℹ️  Metadata         → models\xgboost_20250820-101436.json
✅ Saved 'LightGBM' to models\lightgbm_20250820-101436.joblib
ℹ️  Metadata         → models\lightgbm_20250820-101436.json

📦 Saved models summary:
 - Random Forest: models\random_forest_20250820-101435.joblib
 - XGBoost: models\xgboost_20250820-101436.joblib
 - LightGBM: models\lightgbm_20250820-101436.joblib


In [36]:
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

# =============================
# TOP-2 RECOMMENDATIONS PER USER FOR processed/test_1_2016.csv (SELF-CONTAINED)
# =============================

PRED_DIR = Path('predictions')
PRED_DIR.mkdir(parents=True, exist_ok=True)
TEST_FILE = Path('data/processed/test_1_2016.csv')
OUT_FILE = PRED_DIR / 'test_1_2016_top2.csv'

# Helper to fetch latest saved model

def get_latest_model_path(model_prefix=None, models_dir="models"):
    md = Path(models_dir)
    if not md.exists():
        raise FileNotFoundError("models directory not found. Train and save a model first.")
    candidates = list(md.glob("*.joblib"))
    if model_prefix:
        prefix = model_prefix.lower().replace(" ", "_")
        candidates = [p for p in candidates if p.name.lower().startswith(prefix)]
    if not candidates:
        raise FileNotFoundError("No saved .joblib models found.")
    latest = max(candidates, key=lambda p: p.stat().st_mtime)
    return latest

assert TEST_FILE.exists(), f"Not found: {TEST_FILE}"

print(f"📥 Loading {TEST_FILE} ...")
test_df = pd.read_csv(TEST_FILE)

# Preprocess to match training
print("🧹 Preprocessing to match training...")
test_df = reduce_memory_usage(test_df)
test_df = clean_dataset(test_df)
test_df = filter_data(test_df)
test_df = create_enhanced_features(test_df)

# Build X directly ensuring required columns exist
_needed_num = list(available_num_cols)
_needed_cat = list(available_cat_cols)
for col in _needed_num:
    if col not in test_df.columns:
        test_df[col] = 0
for col in _needed_cat:
    if col not in test_df.columns:
        test_df[col] = 'Unknown'
X_all = test_df[_needed_num + _needed_cat]

# Keep user ids
customer_ids = test_df.loc[X_all.index, 'customer_id'] if 'customer_id' in test_df.columns else pd.Series(np.arange(len(X_all)), index=X_all.index)

# Ensure model is loaded
try:
    loaded_pipeline
except NameError:
    latest_model = get_latest_model_path(None, models_dir='models')
    print(f"📦 Loading latest model → {latest_model}")
    loaded_pipeline = joblib.load(latest_model)

# Predict probabilities in batches and produce top-2
chunk_size = 30000
num_rows = len(X_all)
print(f"\n🔮 Generating top-2 recommendations for {num_rows:,} users...")

recs_rec1 = []
recs_rec2 = []
rec_indices = []

def top2_from_proba(y_proba_batch, tar_cols):
    n_rows = y_proba_batch[0].shape[0]
    out = []
    for i in range(n_rows):
        scores = []
        for j, product in enumerate(tar_cols):
            proba_ij = y_proba_batch[j][i]
            p1 = float(proba_ij[1]) if getattr(proba_ij, 'shape', None) and len(proba_ij) > 1 else float(proba_ij[0])
            scores.append((product, p1))
        scores.sort(key=lambda x: x[1], reverse=True)
        top2 = [p for p, _ in scores[:2]]
        if len(top2) < 2:
            top2 += [''] * (2 - len(top2))
        out.append((top2[0], top2[1]))
    return out

for start in range(0, num_rows, chunk_size):
    end = min(start + chunk_size, num_rows)
    idx = X_all.index[start:end]
    X_batch = X_all.loc[idx]

    try:
        y_proba_batch = loaded_pipeline.predict_proba(X_batch)
        top2_batch = top2_from_proba(y_proba_batch, tar_cols)
    except Exception as e:
        print(f"⚠️ predict_proba unavailable for rows {start}:{end}: {e}. Falling back to labels.")
        y_pred_batch = loaded_pipeline.predict(X_batch)
        top2_batch = []
        for row in y_pred_batch:
            ones = [tar_cols[i] for i, v in enumerate(row) if v == 1]
            zeros = [tar_cols[i] for i, v in enumerate(row) if v == 0]
            ranked = (ones + zeros + ['',''])[:2]
            top2_batch.append((ranked[0], ranked[1]))

    rec1, rec2 = zip(*top2_batch)
    recs_rec1.extend(rec1)
    recs_rec2.extend(rec2)
    rec_indices.extend(list(idx))
    print(f" - Done users {start:,} → {end:,}")

# Assemble output
out_df = pd.DataFrame({
    'customer_id': customer_ids.loc[rec_indices].values,
    'rec1': recs_rec1,
    'rec2': recs_rec2,
}, index=pd.Index(rec_indices, name='index')).reset_index(drop=True)

out_df.to_csv(OUT_FILE, index=False)
print(f"\n✅ Saved top-2 recommendations → {OUT_FILE}")


📥 Loading data\processed\test_1_2016.csv ...
🧹 Preprocessing to match training...
Mem. usage decreased to 987.76 Mb (56.2% reduction)
🧹 CLEANING DATASET...
1️⃣ Handling missing values...
   ✅ Filled payroll_final_label: 0 missing → 0
   ✅ Filled pensions_2_final_label: 0 missing → 0

2️⃣ Creating customer tenure feature...
   ✅ Created 'customer_tenure_days' feature
   📊 Range: -3 to 7682 days
   🗑️ Dropped 'registration_date' column

3️⃣ Converting last_primary_date to binary indicator...
   ✅ Created 'was_primary_customer' binary feature
   📊 6,151,862 nulls → 0, 13,246 dates → 1
   🗑️ Dropped 'last_primary_date' column

4️⃣ Removing constant and duplicate columns...
   🗑️ Dropped 'address_type' (constant value)
   🗑️ Dropped 'province_code' (duplicate of province_name)

5️⃣ Cleaning numeric columns...
   ✅ Cleaned 'age': int16 → numeric (0 nulls)
   ⚠️ Converted 14 negative seniority values to NaN
   ✅ Cleaned 'seniority': int32 → numeric (14 nulls)

📊 CLEANUP SUMMARY:
-------------